In [3]:
import pandas as pd
import sklearn
from sklearn import model_selection
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder

print(sklearn.__version__)

C:\Users\evanh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


1.4.2


In [4]:
pd.set_option('max_columns', None)
pd.set_option('display.max_rows', 40)

OptionError: Pattern matched multiple keys

In [5]:
train = pd.read_csv('train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
train.describe(include = 'all')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,...,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,...,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,...,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,...,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,365.750000,20.000000,NaN,59.000000,7553.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN,129975.000000
50%,730.500000,50.000000,NaN,69.000000,9478.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,1095.250000,70.000000,NaN,80.000000,11601.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


In [7]:
cat_features = train.describe(include = 'all').loc['mean'].isna()
cat_features['MSSubClass'] = True
train[cat_features[cat_features == True].keys()] = train[cat_features[cat_features == True].keys()].astype("category")

In [8]:
print(sklearn.__file__)

C:\Users\evanh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\__init__.py


In [9]:
X = train.loc[:,train.columns != 'SalePrice']
y = train.loc[:,'SalePrice']


X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y)

In [10]:
X_train.dtypes

Id                  int64
MSSubClass       category
MSZoning         category
LotFrontage       float64
LotArea             int64
                   ...   
MiscVal             int64
MoSold              int64
YrSold              int64
SaleType         category
SaleCondition    category
Length: 80, dtype: object

In [12]:
baseline = HistGradientBoostingRegressor(categorical_features="from_dtype")
baseline.fit(X_train,y_train.values.ravel())#

HistGradientBoostingRegressor(categorical_features='from_dtype')

In [13]:
baseline.score(X_test,y_test)

0.9116125949207556

In [15]:
submission = HistGradientBoostingRegressor(categorical_features="from_dtype").fit(X,y)

In [21]:
test = pd.read_csv('test.csv')

cat_features = test.describe(include = 'all').loc['mean'].isna()
cat_features['MSSubClass'] = True
test[cat_features[cat_features == True].keys()] = train[cat_features[cat_features == True].keys()].astype("category")

X = test.loc[:,test.columns != 'SalePrice']


In [32]:
test['SalePrice'] = submission.predict(X)

In [33]:
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,predict,SalePrice
0,1461,60,RL,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,6,2010,WD,Normal,140429.999135,140429.999135
1,1462,20,RL,81.0,14267,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,12500,6,2010,WD,Normal,175719.462335,175719.462335
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,3,2010,WD,Normal,194345.402105,194345.402105
3,1464,70,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,6,2010,WD,Abnorml,187960.086531,187960.086531
4,1465,60,RL,43.0,5005,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,1,2010,WD,Normal,193246.362775,193246.362775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,20,FV,21.0,1936,Pave,Pave,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,6,2006,WD,Normal,118355.255537,118355.255537
1455,2916,60,RL,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2006,WD,Normal,125985.122366,125985.122366
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,9,2006,WD,Normal,178332.095739,178332.095739
1457,2918,70,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,NaN,GdPrv,Shed,700,7,2006,WD,Normal,134330.958228,134330.958228


In [36]:
test[['Id','SalePrice']].to_csv('sub1.csv', index = False)